## [itertools : 高效节省内存的方法](https://mp.weixin.qq.com/s/qAXxSmkroqDItzTCbRlQMA)



## [官方文档 - itertools](https://docs.python.org/3.7/library/itertools.html)

Python的itertools模块提供节省内存的高效迭代器，里面实现基本都借助于生成器，所以一方面了解这12个函数所实现的基本功能，同时也能加深对生成器(generator)的理解，为我们写出更加高效、简洁、漂亮的代码打下坚实基础。

### 1. 拼接元素

itertools 中的 chain 函数实现元素拼接，原型如下，参数 * 表示个数可变的参数
> chain(*iterables)

In [8]:
from itertools import chain

In [9]:
list(chain(["I", "love"], ["python"], ["very", ["much"]]))

['I', 'love', 'python', 'very', ['much']]

它有点 join 的味道，但是比 join 强，它的重点在于参数都是可迭代的实例

那么，chain 如何实现高效节省内存的呢？chain 大概的实现代码如下：
```python
def chain(*iterables):
    for it in iterables:
        for element in it:
            yield element
```
以上代码不难理解，`chain 本质是返回一个生成器，所以它实际上是一次读入一个元素到内存，所以做到最高效地节省内存`

### 2. 逐个累积

返回列表的累积汇总值，原型
> accumulate(iterable, func, *, initial=None)

In [10]:
from itertools import accumulate

In [11]:
list(accumulate([1, 2, 3, 4, 5, 6], lambda x, y : x * y))

[1, 2, 6, 24, 120, 720]

accumulate 大概的实现代码如下：
```python
def accumulate(iterable, func=operator.add, *, initial=None):
    it = iter(iterable)
    total = initial
    if initial is None:
        try:
            total = next(it)
        except StopIteration:
            return
    yield total
    for element in it:
        total = func(total, element)
        yield total
```
与chain简单的yield不同，此处稍微复杂一点，yield有点像return，所以`yield total`那行直接就返回一个元素，也就是iterable的第一个元素，因为任何时候这个函数返回的第一个元素就是它的第一个。又因为yield返回的是一个generator对象，比如名字gen，所以next(gen)时，代码将会执行到`for element in it:`这行，而此时的迭代器it 已经指到iterable的第二个元素，OK，相信你懂了！

### 3. 漏斗筛选

功能类似于漏斗功能，原型：
> compress(data, selectors)

In [17]:
from itertools import compress

In [18]:
# 0 : false
# positive number : true

list(compress('abcdefg', [1, 1, 0, 1]))

['a', 'b', 'd']

compress 返回的元素个数等于两个参数中较短的列表长度
```python
def compress(data, selectors):
    return (d for d, s in zip(data, selectors) is s)
```

### 4. 段位筛选

扫描列表，从不满足条件处开始往后保留，原型如下：
> dropwhile(predicate, iterable)

其实现的大概代码如下：
```python
def dropwhile(predicate, iterable):
    iterable = iter(iterable)
    for x in iterable:
        if not predicate(x):
            yield x
            break
    for x in iterable:
        yield x
```

In [20]:
from itertools import dropwhile

In [21]:
list(dropwhile(lambda x : x < 3, [1, 0, 2, 4, 1, 1, 3, 5, -5]))

[4, 1, 1, 3, 5, -5]

### 5. 段位筛选2

扫描列表，只要满足条件就从可迭代对象中返回元素，直到不满足条件为止，原型如下：
> takewhile(predictate, iterable)

实现它的大概代码如下：
```python
def takewhile(predicate, iterable):
    for x in iterable:
        if predicate(x):
            yield x
        else:
            break  # 程序中断
```

In [2]:
from itertools import takewhile

In [4]:
list(takewhile(lambda x: x < 5, [1, 4, 6, 4, 1]))

[1, 4]

In [7]:
# 只要满足条件就从可迭代对象中返回元素，直到不满足条件为止

list(takewhile(lambda x: x < 5, [1, 3, 5, 2, 6]))

[1, 3]

### 6. 次品筛选

扫描列表，只要不满足条件都保留，原型如下：
> dropwhile(predicate, iterable)

实现它的大概代码如下：
```python
def filterfalse(predicate, iterable):
    # filterfalse(lambda x: x%2, range(10)) --> 0 2 4 6 8
    if predicate is None:
        predicate = bool
    for x in iterable:
        if not predicate(x):
            yield x
```

In [5]:
from itertools import filterfalse

In [6]:
list(filterfalse(lambda x: x%2 == 0, [-1, 0, 1, 2, 3, 4, 1, -2]))

[-1, 1, 3, 1]

### 7. 切片筛选

相比于 list\[: 1\], `islice`更节省内存，原型如下：
> islice(iterable, start, stop\[, step\])

```python
def islice(iterable, *args):
    s = slice(*args)
    start, stop, step = s.start or 0, s.stop or sys.maxsize, s.step or 1
    it = iter(range(start, stop, step))
    try:
        nexti = next(it)
    except StopIteration:
        for i, element in zip(range(start), iterable):
            pass
        return
    try:
        for i, element in enumberate(iterable):
            if i == nexti:
                yield element
                nexti = next(it)
    except StopIteration:
        for i, element in zip(range(i + 1, stop), iterable):
            pass        

```
巧妙利用生成器迭代结束时会抛出异常StopIteration，做一些边界处理的事情。

In [23]:
from itertools import islice

In [26]:
# iterable : 'abcdefg', start : 1, stop : 4, step : 2

list(islice('abcdefg', 1, 4, 2))

['b', 'd']

### 6. 细胞分裂
tee 函数类似于我们熟知的细胞分裂，它能复制原迭代器 n 个，原型如下：
> tee(iterable, n = 2)

```python
def tee(iterable, n = 2):
    it = iter(iterable)
    deques = [collections.deque() for i in range(n)]
    def gen(mydeque):
        while True:
            if not mydeque:
                try:
                    newval = next(it)
                except StopIteration:
                    return
                for d in deques:
                    d.append(newval)
            yield mydeque.popleft()
    return tuple(gen(d) for d in deques)
                    
```
tee 实现内部使用一个队列类型deques，起初生成空队列，向复制出来的每个队列中添加元素newval, 同时yield 当前被调用的mydeque中的最左元素。

In [29]:
from itertools import tee

In [32]:
a = tee([1, 4, 6, 4, 1], 2)

复制出的迭代器是独立的

In [33]:
print(next(a[0]))
print(next(a[0]))
print(next(a[0]))

1
4
6


In [34]:
print(next(a[1]))
print(next(a[1]))
print(next(a[1]))

1
4
6


### 9. map 变体

starmap 可以看作是 map 的变体，它能更加节省内存，同时 iterable 的元素必须也为可迭代对象，原型如下：
> starmap(function, iterable)

```python
def starmap(function, iterable):
    for args in iterable:
        yield function(*args)
```

In [35]:
from itertools import starmap

In [37]:
list(starmap(lambda x, y : str(x) + '-' + str(y), [('a', 1), ('b', 2), ('c', 3)]))

['a-1', 'b-2', 'c-3']

### 10. 复制元素

repeat 实现复制元素 n 次，原型如下：
> repeat(object\[, times\])

它的实现细节大概如下：
```python
def repeat(object, times=None):
    if times is None:  # 如果 times 不设置，将一起 repeat 下去
        while True:
            yield object
    else:
        for i in range(times):
            yield object
```

In [1]:
from itertools import repeat

In [2]:
list(repeat(6, 3))

[6, 6, 6]

### 11. 笛卡尔积
笛卡尔积实现的效果同下：
> ((x, y) for x in A for y in B)

它的实现细节：
```python
def product(*args, repeat=1):
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x + [y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)
```

In [3]:
from itertools import product

In [4]:
list(product("ABCD", "xy"))

[('A', 'x'),
 ('A', 'y'),
 ('B', 'x'),
 ('B', 'y'),
 ('C', 'x'),
 ('C', 'y'),
 ('D', 'x'),
 ('D', 'y')]

### 12. 加强版 zip
组合值。若可迭代对象的长度未对齐，将根据 `fillvalue` 填充缺失值，注意：`迭代持续到耗光最长的可迭代对象`

实现细节：
```python
def zip_longest(*args, fillvalue=None):
    num_active = len(iterators)
    if not num_active:
        return
    while True:
        values = []
        for i it in enumerate(iterators):
            try:
                value = next(it)
            except StopIteraton:
                num_active -= 1
                if not num_active:
                    return
                iterators[i] = repeat(fillvalue)
                value = fillvalue
            values.append(value)
        yield tuple(values)
                
```

In [6]:
from itertools import zip_longest

In [8]:
list(zip_longest('ABCD', 'xy', fillvalue='-'))

[('A', 'x'), ('B', 'y'), ('C', '-'), ('D', '-')]

它里面使用repeat，也就是在可迭代对象的长度未对齐时，根据 fillvalue 填充缺失值。

理解上面代码的关键是迭代器对象(iter)，next方法的特殊性：

In [10]:
for i, it in enumerate([iter([1, 2, 3]), iter(['x', 'y'])]):
    print(next(it))

1
x
